In [10]:
import pandas as pd
import torch
import numpy as np
from transformers import BertTokenizer, BertModel
from torch import nn
from torch.optim import Adam
from tqdm import tqdm
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import csv
from sklearn.utils import shuffle
import tensorflow as tf
import pandas as pd
import torch
import numpy as np
from transformers import BertTokenizer, BertModel , AutoModel
from torch import nn
from torch.optim import Adam
from tqdm import tqdm
import os 
import string 
import glob
import joblib
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize 
from nltk.corpus import stopwords

In [11]:
dataset = pd.read_pickle("dataframefr_pldout.pkl")
dataset2 = pd.read_pickle("dataframefr_eng_pldout.pkl")

print(len(dataset))
print(len(dataset2))

2854
2854


In [12]:
test_dataset = pd.read_pickle("testfrancetranslatetoromanian.pkl")
print(test_dataset)

test_dataset = pd.DataFrame(test_dataset)
test_dataset['label']= test_dataset['label'].apply(lambda x:1 if x == 'satire' else 0)

                                                content      label
0     [Hello circumspect doctor little chamomile yea...     satire
1     [sydney rare genetic diseases of which media r...     satire
2     [4th Directive on driving permits p2a double s...     satire
3     [Hello doctor little elise call commonly gifte...     satire
4     [has sold more jk rowling harry potter known a...     satire
...                                                 ...        ...
2849  [invited Wednesday morning bfmtv alexis corbiè...  nonSatire
2850  [point mayors elected from the first round mun...  nonSatire
2851  [this consoleracy non companion girl na took p...  nonSatire
2852  [europe ecologies Greens achieved perfect coup...  nonSatire
2853  [heart debate easily caricatural question abol...  nonSatire

[2854 rows x 2 columns]


In [13]:
dataset

,Pooled output france
0,"[[0.0986732691526413, 0.393089234828949, -0.07..."
1,"[[0.12457945197820663, 0.4128280282020569, 0.0..."
2,"[[0.23128312826156616, 0.47273799777030945, 0...."
3,"[[0.1423369199037552, 0.37577834725379944, 0.0..."
4,"[[0.09319072216749191, 0.43242374062538147, 0...."
...,...
2849,"[[0.4619225263595581, -0.004393668845295906, 0..."
2850,"[[0.5446183085441589, -0.02991306595504284, 0...."
2851,"[[0.4565497636795044, -0.042465463280677795, 0..."
2852,"[[0.4682226777076721, -0.08964677900075912, 0...."


In [15]:
dataset2

,Pooled output france-eng
0,"[[-0.5763624906539917, -0.32024598121643066, -..."
1,"[[-0.740075409412384, -0.052671320736408234, 0..."
2,"[[-0.7120029330253601, -0.132234126329422, 0.1..."
3,"[[-0.6213632225990295, -0.1550714075565338, 0...."
4,"[[-0.6734880208969116, -0.3700582683086395, -0..."
...,...
2849,"[[-0.8730265498161316, -0.08605591952800751, -..."
2850,"[[-0.6107222437858582, 0.022510454058647156, -..."
2851,"[[-0.8401784896850586, -0.11244022101163864, -..."
2852,"[[-0.6963432431221008, 0.1536889225244522, -0...."


In [16]:
dataset["Pooled output france"][0][0]

[0.0986732691526413,
 0.393089234828949,
 -0.07619977742433548,
 0.13540305197238922,
 0.6210857629776001,
 -0.32825958728790283,
 -0.4250953793525696,
 0.09531806409358978,
 0.8683711290359497,
 -0.5902928709983826,
 -0.5722011923789978,
 -0.29185256361961365,
 -0.34717264771461487,
 0.566238284111023,
 0.09162022918462753,
 0.5382823348045349,
 -0.22248294949531555,
 -0.43120190501213074,
 -0.4996376037597656,
 0.14624576270580292,
 0.310494989156723,
 -0.6130580306053162,
 0.8368504643440247,
 0.6114107966423035,
 0.39573150873184204,
 0.4889828562736511,
 -0.25992751121520996,
 -0.21127407252788544,
 0.5838837623596191,
 -0.5017033219337463,
 -0.7326120734214783,
 -0.8088769316673279,
 -0.6381609439849854,
 0.2906915843486786,
 -0.16859731078147888,
 0.42485737800598145,
 -0.6628500819206238,
 0.5620782375335693,
 0.2587994337081909,
 0.5412477254867554,
 0.669015645980835,
 0.05162596330046654,
 -0.6640283465385437,
 -0.8381609320640564,
 0.5536924004554749,
 0.6405285000801086,
 

In [17]:
dataset3 = np.concatenate((dataset["Pooled output france"][0][0] , dataset2["Pooled output france-eng"][0][0]))

In [18]:
len(dataset)

2854

In [19]:
len(dataset2)

2854

In [20]:
france= []
for i in range(len(dataset)):
    con = np.concatenate((dataset["Pooled output france"][i][0] , dataset2["Pooled output france-eng"][i][0]))
    france.append(con)
        
        

In [21]:
len(france)

2854

In [39]:
len(france[2])


1536

In [26]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import datasets
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression


X, y = france, test_dataset['label']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1234)


clf = LogisticRegression(solver='lbfgs', max_iter=100)
clf.fit(X_train,y_train)
y_pred = clf.predict(X_test)

def accuracy(y_pred, y_test):
    return np.sum(y_pred==y_test)/len(y_test)

acc = accuracy(y_pred, y_test)
print(acc)

0.9877408056042032


/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [27]:
print("X train",X_train[0])
print("y train",y_train[0])

X train [ 0.11586583  0.52629745  0.15717466 ...  0.82912958  0.86299253
 -0.5662033 ]
y train 1


In [28]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import datasets
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression


X, y = france, test_dataset['label']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1234)

clf = LogisticRegression(max_iter=35)
clf.fit(X_train,y_train)
y_pred = clf.predict(X_test)

def accuracy(y_pred, y_test):
    return np.sum(y_pred==y_test)/len(y_test)

acc = accuracy(y_pred, y_test)
print(acc)

0.989492119089317


/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [57]:
np.array(france)

array([[ 0.09867327,  0.39308923, -0.07619978, ...,  0.38283619,
         0.68834621, -0.80133677],
       [ 0.12457945,  0.41282803,  0.01997186, ...,  0.8555274 ,
         0.77834785, -0.517892  ],
       [ 0.23128313,  0.472738  ,  0.0682683 , ...,  0.77802187,
         0.88888913, -0.61540461],
       ...,
       [ 0.45654976, -0.04246546,  0.24994993, ..., -0.21727836,
        -0.46156871,  0.25781849],
       [ 0.46822268, -0.08964678,  0.24941586, ...,  0.34175906,
        -0.57965231,  0.77978367],
       [ 0.50689209, -0.06059765,  0.2838929 , ...,  0.07774515,
         0.75215763, -0.75968534]])

In [54]:
np.array(test_dataset['label'])

array([1, 1, 1, ..., 0, 0, 0])

In [59]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression

x = np.array([[0.09867327, 0.39308923, -0.07619978, ..., 0.38283619, 0.68834621, -0.80133677],
              [0.12457945, 0.41282803, 0.01997186, ..., 0.8555274, 0.77834785, -0.517892],
              [0.23128313, 0.472738, 0.0682683, ..., 0.77802187, 0.88888913, -0.61540461],
              ...,
              [0.45654976, -0.04246546, 0.24994993, ..., -0.21727836, -0.46156871, 0.25781849],
              [0.46822268, -0.08964678, 0.24941586, ..., 0.34175906, -0.57965231, 0.77978367],
              [0.50689209, -0.06059765, 0.2838929, ..., 0.07774515, 0.75215763, -0.75968534]])

# Assuming y is the corresponding labels
y = np.array([0, 0, 1, ..., 1, 0, 1])

# Create and fit the logistic regression model
model = LogisticRegression()
model.fit(x, y)

# Generate a set of points for plotting the decision boundary
x_min, x_max = x[:, 0].min() - 1, x[:, 0].max() + 1
y_min, y_max = x[:, 1].min() - 1, x[:, 1].max() + 1
xx, yy = np.meshgrid(np.arange(x_min, x_max, 0.02),
                     np.arange(y_min, y_max, 0.02))
Z = model.predict(np.c_[xx.ravel(), yy.ravel()])
Z = Z.reshape(xx.shape)

# Plot the data points and decision boundary
plt.contourf(xx, yy, Z, alpha=0.8)
plt.scatter(x[:, 0], x[:, 1], c=y, edgecolors='k')
plt.xlabel('Feature 1')
plt.ylabel('Feature 2')
plt.title('Logistic Regression Decision Boundary')
plt.show()


/var/folders/ph/prmd5g8j6vs8gsdh6hrmfy8h0000gq/T/ipykernel_15839/3166910531.py:5: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  x = np.array([[0.09867327, 0.39308923, -0.07619978, ..., 0.38283619, 0.68834621, -0.80133677],


ValueError: setting an array element with a sequence.